**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Dec  5 13:07:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'사무부 계좌 개설 서류'
'14차 단과대운영위원회 회의록.hwp'
'붙임1. 제안대회 신청서.docx'
'2020학년도 1학기 기말과제_앱창작실습.gdoc'
 20210610_162823.jpg
'2021 SW융합대학 블루밍학생회 11월 가예산안.gsheet'
'2021 SW융합대학 블루밍학생회 12월 가예산안-1.gsheet'
'2021 SW융합대학 블루밍학생회 8월 가예산안-1-1.csv'
'붙임 2. 제10회 빅데이터기반 성남시 청소년 정책제안대회 신청서 및 제안서.docx'
'붙임2. 제안대회 제안서.docx'
 32194061_정민균_10주차_개별보고서.docx
 32194061_정민균_11주차_개별보고서.docx
 32194061_정민균_12주차_개별보고서.docx
 32194061_정민균_13주차_개별보고서.docx
 32194061_정민균_14주차_개별보고서.docx
 32194061정민균-1.py
 32194061정민균-2.py
'32194061 정민균-3.py'
 32194061_정민균_9주차_개별보고서.docx
 32194061_정민균_개별보고서.docx
 32194061_정민균.zip
 Classroom
'Colab Notebooks'
'제목 없는 문서.gdoc'
'실현 및 구현 방안.gdoc'

**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15376, done.
remote: Total 15376 (delta 0), reused 0 (delta 0), pack-reused 15376
Receiving objects: 100% (15376/15376), 14.01 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (10339/10339), done.


# 새 섹션

**2) Compile Darknet using Nvidia GPU**


In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [6]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp /mydrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg

In [7]:
!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /mydrive/yolov3/data/obj.names data/obj.names
!cp /mydrive/yolov3/data/obj.data data/obj.data
!mkdir data/obj

In [8]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-05 13:09:13--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  56.1MB/s    in 2.8s    

2021-12-05 13:09:16 (56.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [9]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/100.jpg        
  inflating: data/obj/100.txt        
  inflating: data/obj/101.jpg        
  inflating: data/obj/101.txt        
  inflating: data/obj/102.jpg        
  inflating: data/obj/102.txt        
  inflating: data/obj/11.png         
  inflating: data/obj/11.txt         
  inflating: data/obj/12.jpg         
  inflating: data/obj/12.txt         
  inflating: data/obj/13.jpg         
  inflating: data/obj/13.txt         
  inflating: data/obj/14.jpg         
  inflating: data/obj/14.txt         
  inflating: data/obj/15.jpg         
  inflating: data/obj/15.txt         
  inflating: data/obj/16.jpg         
  inflating: data/obj/16.txt         
  inflating: data/obj/17.jpeg        
  inflating: data/obj/17.txt         
  inflating: data/obj/18.jpeg        
  inflating: data/obj/18.txt         
  inflating: data/obj/19.jpg         
  inflating: data/obj/19.txt         
  inflating: data/obj/20.jpg         
  inflating: 

In [10]:
import glob
images_list = glob.glob('data/obj/*.jpg')
#images_list = glob.glob("data/obj/*.png")
print(images_list)
len(images_list)


['data/obj/33.jpg', 'data/obj/78.jpg', 'data/obj/102.jpg', 'data/obj/58.jpg', 'data/obj/68.jpg', 'data/obj/101.jpg', 'data/obj/30.jpg', 'data/obj/100.jpg', 'data/obj/57.jpg', 'data/obj/65.jpg', 'data/obj/28.jpg', 'data/obj/73.jpg', 'data/obj/85.jpg', 'data/obj/97.jpg', 'data/obj/43.jpg', 'data/obj/75.jpg', 'data/obj/98.jpg', 'data/obj/64.jpg', 'data/obj/92.jpg', 'data/obj/89.jpg', 'data/obj/eight.jpg', 'data/obj/45.jpg', 'data/obj/91.jpg', 'data/obj/14.jpg', 'data/obj/road.jpg', 'data/obj/87.jpg', 'data/obj/96.jpg', 'data/obj/16.jpg', 'data/obj/19.jpg', 'data/obj/20.jpg', 'data/obj/seven.jpg', 'data/obj/72.jpg', 'data/obj/42.jpg', 'data/obj/59.jpg', 'data/obj/13.jpg', 'data/obj/31.jpg', 'data/obj/69.jpg', 'data/obj/99.jpg', 'data/obj/44.jpg', 'data/obj/66.jpg', 'data/obj/60.jpg', 'data/obj/81.jpg', 'data/obj/74.jpg', 'data/obj/82.jpg', 'data/obj/77.jpg', 'data/obj/93.jpg', 'data/obj/67.jpg', 'data/obj/70.jpg', 'data/obj/53.jpg', 'data/obj/94.jpg', 'data/obj/51.jpg', 'data/obj/76.jpg', 

70

In [11]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [13]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 8, batch = 8, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho